In [57]:
import torch.nn as nn
import torch.optim as optim

from train import *
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### main

In [58]:
# load params
model_dir = 'experiments/base_model'
json_path = os.path.join(model_dir, 'params.json')
params = utils.Params(json_path)

In [59]:
params.dict

{'learning_rate': 0.001,
 'batch_size': 64,
 'num_epochs': 10,
 'seq_len': 10,
 'lstm_hidden_dim': 32,
 'embedding_dim': 50,
 'n_layers': 2,
 'dropout': 0.5,
 'save_summary_steps': 100,
 'cuda': 0,
 'vocab_size': 2697,
 'number_of_tags': 2}

In [60]:
# load data
data_dir = 'data/5w'
data_loader = DataLoader(data_dir, params)
data = data_loader.load_data(['train', 'val'], data_dir)
train_data = data['train']
val_data = data['val']

In [61]:
train_data.keys()

dict_keys(['data', 'labels', 'size'])

In [62]:
len(train_data['data']), len(train_data['labels']), train_data['size']

(16383, 16383, 16383)

In [63]:
# specify the train and val dataset sizes
params.train_size = train_data['size']
params.val_size = val_data['size']

In [64]:
params.dict

{'learning_rate': 0.001,
 'batch_size': 64,
 'num_epochs': 10,
 'seq_len': 10,
 'lstm_hidden_dim': 32,
 'embedding_dim': 50,
 'n_layers': 2,
 'dropout': 0.5,
 'save_summary_steps': 100,
 'cuda': 0,
 'vocab_size': 2697,
 'number_of_tags': 2,
 'train_size': 16383,
 'dev_size': 2048,
 'test_size': 2048,
 'pad_word': '<pad>',
 'unk_word': 'UNK',
 'val_size': 2048}

In [65]:
# define the model and optimizer
model = net.ReviewRNN(params)
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)

In [66]:
# fetch loss function and metrics
loss_fn = nn.CrossEntropyLoss()  # net.loss_fn
metrics = net.metrics

### train

In [67]:
model.train()

ReviewRNN(
  (embedding): Embedding(2697, 50)
  (lstm): LSTM(50, 32, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

In [68]:
data_iterator = data_loader.data_iterator(train_data, params)

In [80]:
summary = []
loss_avg = utils.RunningAverage()

In [81]:
for i in range(100):
        # (1) get data
        train_batch, labels_batch = next(data_iterator)

        # (2) standard optimization steps
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # (3) evaluate summaries only once in a while
        if i % 10 == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.numpy()
            labels_batch = labels_batch.data.numpy()

            # compute all metrics on this batch
            summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.item()
            summary.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.item())

In [82]:
summary

[{'accuracy': 0.875, 'loss': 0.5238780379295349},
 {'accuracy': 0.875, 'loss': 0.4653232991695404},
 {'accuracy': 0.78125, 'loss': 0.495085746049881},
 {'accuracy': 0.65625, 'loss': 0.6485917568206787},
 {'accuracy': 0.9375, 'loss': 0.3032645881175995},
 {'accuracy': 0.328125, 'loss': 1.1923974752426147},
 {'accuracy': 0.828125, 'loss': 0.4467722177505493},
 {'accuracy': 0.90625, 'loss': 0.3402827978134155},
 {'accuracy': 0.890625, 'loss': 0.32180944085121155},
 {'accuracy': 0.859375, 'loss': 0.3829444646835327}]

This involved formula just computes average accuracy and loss in summary when the loop is finished.

In [85]:
metrics_mean = {metric: np.mean([x[metric] for x in summary]) for metric in summary[0]}

In [86]:
metrics_mean

{'accuracy': 0.79375, 'loss': 0.5120349824428558}

In [87]:
metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())

In [88]:
metrics_string

'accuracy: 0.794 ; loss: 0.512'